In [17]:
import pandas as pd
import os
import glob

In [18]:
# Load all street crime files
street_files = glob.glob('../data/raw/**/*street*.csv', recursive=True)

dfs = []
for file in street_files:
    df = pd.read_csv(file)
    # Extract force name from filename
    df['force'] = os.path.basename(file).split('-')[2]
    dfs.append(df)

street = pd.concat(dfs, ignore_index=True)
print(f"Loaded {len(street):,} rows from {len(street_files)} files")
street.head()

Loaded 3,442,941 rows from 72 files


,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context,force
0,4a14d4745da0a2219ecf913fdc28a0c84ae8408954cac6...,2023-01,City of London Police,City of London Police,-0.106220,51.518275,On or near B500,E01000916,Camden 027B,Other theft,Status update unavailable,NaN,city
1,e6e32581c99c5b0f46e5a4d6343e213df349d7069e847d...,2023-01,City of London Police,City of London Police,-0.107010,51.521480,On or near Saffron Street,E01000916,Camden 027B,Other crime,Status update unavailable,NaN,city
2,7b7cb8e7debe8b0ec1637e7cb1dad832cea4eba16c5f52...,2023-01,City of London Police,City of London Police,-0.110350,51.518090,On or near Holborn,E01000917,Camden 027C,Theft from the person,Investigation complete; no suspect identified,NaN,city
3,f7fc44e1e76332f0f575b788522329e6f3ce566fd7472d...,2023-01,City of London Police,City of London Police,-0.107682,51.517786,On or near B521,E01000917,Camden 027C,Other crime,Status update unavailable,NaN,city
4,8083dafd1770af1afca2320c13cbae2420bd2877d9ef29...,2023-01,City of London Police,City of London Police,-0.111596,51.518281,On or near Chancery Lane,E01000914,Camden 028B,Other theft,Status update unavailable,NaN,city


In [19]:
# Drop columns that aren't useful
street = street.drop(columns=['Context', 'Falls within', 'Reported by'], errors='ignore')

# Rename columns to be more code-friendly
street.columns = [c.lower().replace(' ', '_') for c in street.columns]

# Drop rows with no location data
street = street.dropna(subset=['latitude', 'longitude'])

# Convert month to datetime
street['month'] = pd.to_datetime(street['month'])

print(f"Clean dataset: {len(street):,} rows")
print(street.columns.tolist())
street.head()

Clean dataset: 3,416,295 rows
['crime_id', 'month', 'longitude', 'latitude', 'location', 'lsoa_code', 'lsoa_name', 'crime_type', 'last_outcome_category', 'force']


,crime_id,month,longitude,latitude,location,lsoa_code,lsoa_name,crime_type,last_outcome_category,force
0,4a14d4745da0a2219ecf913fdc28a0c84ae8408954cac6...,2023-01-01,-0.106220,51.518275,On or near B500,E01000916,Camden 027B,Other theft,Status update unavailable,city
1,e6e32581c99c5b0f46e5a4d6343e213df349d7069e847d...,2023-01-01,-0.107010,51.521480,On or near Saffron Street,E01000916,Camden 027B,Other crime,Status update unavailable,city
2,7b7cb8e7debe8b0ec1637e7cb1dad832cea4eba16c5f52...,2023-01-01,-0.110350,51.518090,On or near Holborn,E01000917,Camden 027C,Theft from the person,Investigation complete; no suspect identified,city
3,f7fc44e1e76332f0f575b788522329e6f3ce566fd7472d...,2023-01-01,-0.107682,51.517786,On or near B521,E01000917,Camden 027C,Other crime,Status update unavailable,city
4,8083dafd1770af1afca2320c13cbae2420bd2877d9ef29...,2023-01-01,-0.111596,51.518281,On or near Chancery Lane,E01000914,Camden 028B,Other theft,Status update unavailable,city


In [20]:
os.makedirs('../data/processed', exist_ok=True)
street.to_csv('../data/processed/street_clean.csv', index=False)
print("Saved to data/processed/street_clean.csv")

Saved to data/processed/street_clean.csv


In [21]:
street = pd.read_csv('../data/processed/street_clean.csv')
print(street['force'].value_counts())
print(f"\nTotal rows: {len(street):,}")
print(f"Date range: {street['month'].min()} to {street['month'].max()}")

force
metropolitan    3390516
city              25779
Name: count, dtype: int64

Total rows: 3,416,295
Date range: 2023-01-01 to 2025-12-01
